# Query Expansion technique

Find the synonyms of the words in the user query to expand the query to yield better results

#### Better query -> Better retrieval -> Better grounding for LLM -> Better answers

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [3]:
## Step 1: Load and split the data
loader=TextLoader("langchain_crewai_dataset.txt")
raw_docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks=splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using st

In [4]:
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)
retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":5})

In [5]:
# Step 4: LLM and prompt for query expansion

llm=init_chat_model("groq:llama-3.1-8b-instant")    

In [9]:
query_expansion_prompt=PromptTemplate.from_template(
    """You are a helpful assistant. Expand the following query to import document retrieval by adding relevant synonyms, technical terms and useful context
     
     Original query: "{query}" 
     
     Expanded query: 
     """
)
query_expanasion_chain=query_expansion_prompt | llm | StrOutputParser()

In [10]:
query_expanasion_chain.invoke({"query":"Langchain memory"})


'Here\'s an expanded query for importing document retrieval related to "Langchain memory":\n\n**Expanded Query:**\n\n("Langchain memory" OR "Memory-based Langchain" OR "Langchain knowledge base" OR "Knowledge graph" OR "Memory module" OR "Memory network" OR "Knowledge retrieval" OR "Document retrieval" OR "Question answering" OR "Conversational AI" OR "Large language models" OR "LLMs" OR "Memory-augmented neural networks" OR "MANNs") \n\n**Context:**\n\n- Langchain is an open-source AI framework that enables the development of conversational AI systems.\n- Memory-based Langchain refers to the use of memory modules to store and retrieve knowledge for conversational AI systems.\n- Knowledge base and knowledge graph are related concepts that refer to the storage and organization of knowledge in a structured way.\n- Memory module and memory network refer to the components of a system that store and retrieve information from memory.\n- Document retrieval and question answering are related t

In [13]:
# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
                                             Answer the question below based on the context.
                                             
                                             Context: {context}
                                             
                                             Question: {input} """)

document_chain=create_stuff_documents_chain(llm=llm, prompt=answer_prompt)

In [14]:
rag_pipeline = (
    RunnableMap({
        "input": lambda x: x['input'],
        "context": lambda x: retriever.invoke(query_expanasion_chain.invoke({"query": x["input"]}))
    })
    | document_chain
)

In [18]:
query={"input": "What types of memory does LangChain support?"}
print(query_expanasion_chain.invoke({"query":query}))
response=rag_pipeline.invoke(query)
print(f"Answer: {response}")


To expand the query, we'll add relevant synonyms, technical terms, and useful context. Here's the expanded query:

```sql
{
  'input': [
    'What types of memory does LangChain support?',
    'What kinds of memory are supported by LangChain?',
    'Which memory frameworks is LangChain compatible with?',
    'What memory storage options does LangChain provide?',
    'Does LangChain support in-memory computing?',
    'Can LangChain utilize GPU memory?',
    'What types of data can be stored in LangChain memory?',
    'Is LangChain compatible with relational databases as memory storage?',
    'Can LangChain leverage distributed memory?',
    'What are the memory-related features in LangChain?'
  ]
}
```

Additionally, we can include relevant technical terms to help narrow down the search results:

```sql
{
  'input': [..., 'memory storage', 'in-memory computing', 'GPU memory', 'distributed memory', 'relational databases', 'memory management', 'caching', 'data storage'],
  'technical_term